# 実践演習 Day 1：streamlitとFastAPIのデモ
このノートブックでは以下の内容を学習します。

- 必要なライブラリのインストールと環境設定
- Hugging Faceからモデルを用いたStreamlitのデモアプリ
- FastAPIとngrokを使用したAPIの公開方法

演習を始める前に、HuggingFaceとngrokのアカウントを作成し、
それぞれのAPIトークンを取得する必要があります。


演習の時間では、以下の3つのディレクトリを順に説明します。

1. 01_streamlit_UI
2. 02_streamlit_app
3. 03_FastAPI

2つ目や3つ目からでも始められる様にノートブックを作成しています。

復習の際にもこのノートブックを役立てていただければと思います。

### 注意事項
「02_streamlit_app」と「03_FastAPI」では、GPUを使用します。

これらを実行する際は、Google Colab画面上のメニューから「編集」→ 「ノートブックの設定」

「ハードウェアアクセラレーター」の項目の中から、「T4 GPU」を選択してください。

このノートブックのデフォルトは「CPU」になっています。

---

# 環境変数の設定（1~3共有）


GitHubから演習用のコードをCloneします。

In [1]:
!git clone https://github.com/niikun/lecture-ai-engineering.git

Cloning into 'lecture-ai-engineering'...
remote: Enumerating objects: 216, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 216 (delta 59), reused 62 (delta 40), pack-reused 112 (from 3)
Receiving objects: 100% (216/216), 122.97 KiB | 11.18 MiB/s, done.
Resolving deltas: 100% (93/93), done.


必要なAPIトークンを.envに設定します。

「lecture-ai-engineering/day1」の配下に、「.env_template」ファイルが存在しています。

隠しファイルのため表示されていない場合は、画面左側のある、目のアイコンの「隠しファイルの表示」ボタンを押してください。

「.env_template」のファイル名を「.env」に変更します。「.env」ファイルを開くと、以下のような中身になっています。


```
HUGGINGFACE_TOKEN="hf-********"
NGROK_TOKEN="********"
```
ダブルクオーテーションで囲まれた文字列をHuggingfaceのアクセストークンと、ngrokの認証トークンで書き変えてください。

それぞれのアカウントが作成済みであれば、以下のURLからそれぞれのトークンを取得できます。

- Huggingfaceのアクセストークン
https://huggingface.co/docs/hub/security-tokens

- ngrokの認証トークン
https://dashboard.ngrok.com/get-started/your-authtoken

書き換えたら、「.env」ファイルをローカルのPCにダウンロードしてください。

「01_streamlit_UI」から「02_streamlit_app」へ進む際に、CPUからGPUの利用に切り替えるため、セッションが一度切れてしまいます。

その際に、トークンを設定した「.env」ファイルは再作成することになるので、その手間を減らすために「.env」ファイルをダウンロードしておくと良いです。

「.env」ファイルを読み込み、環境変数として設定します。次のセルを実行し、最終的に「True」が表示されていればうまく読み込めています。

In [2]:
!pip install python-dotenv
from dotenv import load_dotenv, find_dotenv

%cd /content/lecture-ai-engineering/day1
load_dotenv(find_dotenv())

/content/lecture-ai-engineering/day1


True

# 01_streamlit_UI

ディレクトリ「01_streamlit_UI」に移動します。

In [3]:
%cd /content/lecture-ai-engineering/day1/01_streamlit_UI

/content/lecture-ai-engineering/day1/01_streamlit_UI


必要なライブラリをインストールします。

In [4]:
%%capture
!pip install -r requirements.txt

ngrokのトークンを使用して、認証を行います。

In [ ]:
!ngrok authtoken $$NGROK_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


アプリを起動します。

In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(8501).public_url
print(f"公開URL: {public_url}")
!streamlit run app.py

公開URLの後に記載されているURLにブラウザでアクセスすると、streamlitのUIが表示されます。

app.pyのコメントアウトされている箇所を編集することで、UIがどの様に変化するか確認してみましょう。

streamlitの公式ページには、ギャラリーページがあります。

streamlitを使うとpythonという一つの言語であっても、様々なUIを実現できることがわかると思います。

https://streamlit.io/gallery

後片付けとして、使う必要のないngrokのトンネルを削除します。

In [ ]:
from pyngrok import ngrok
ngrok.kill()

# 02_streamlit_app


ディレクトリ「02_streamlit_app」に移動します。

In [ ]:
%cd /content/lecture-ai-engineering/day1/02_streamlit_app

/content/lecture-ai-engineering/day1/02_streamlit_app


必要なライブラリをインストールします。

In [ ]:
%%capture
!pip install -r requirements.txt

ngrokとhuggigfaceのトークンを使用して、認証を行います。

In [ ]:
!ngrok authtoken $$NGROK_TOKEN
!huggingface-cli login --token $$HUGGINGFACE_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `agent` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `agent`


stramlitでHuggingfaceのトークン情報を扱うために、streamlit用の設定ファイル（.streamlit）を作成し、トークンの情報を格納します。

In [ ]:
# .streamlit/secrets.toml ファイルを作成
import os
import toml

# 設定ファイルのディレクトリ確保
os.makedirs('.streamlit', exist_ok=True)

# 環境変数から取得したトークンを設定ファイルに書き込む
secrets = {
    "huggingface": {
        "token": os.environ.get("HUGGINGFACE_TOKEN", "")
    }
}

# 設定ファイルを書き込む
with open('.streamlit/secrets.toml', 'w') as f:
    toml.dump(secrets, f)

アプリを起動します。

02_streamlit_appでは、Huggingfaceからモデルをダウンロードするため、初回起動には2分程度時間がかかります。

この待ち時間を利用して、app.pyのコードを確認してみましょう。

### Gemma 2

In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(8501).public_url
print(f"公開URL: {public_url}")
!streamlit run app.py

公開URL: https://2c8c-34-87-46-174.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.87.46.174:8501

NLTK loaded successfully.
2025-04-19 13:29:59.179240: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-19 13:29:59.196762: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745069399.222423    4505 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745069399.229935    4505 cuda_blas.cc:1418] Unable to regis

### Phi 4

In [ ]:
!streamlit cache clear

In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(8501).public_url
print(f"公開URL: {public_url}")
!streamlit run app_phi.py

公開URL: https://81b8-34-87-46-174.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.87.46.174:8501

NLTK loaded successfully.
2025-04-19 13:35:18.469716: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-19 13:35:18.486743: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745069718.512901    6122 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745069718.520654    6122 cuda_blas.cc:1418] Unable to regis

アプリケーションの機能としては、チャット機能や履歴閲覧があります。

これらの機能を実現するためには、StreamlitによるUI部分だけではなく、SQLiteを使用したチャット履歴の保存やLLMのモデルを呼び出した推論などの処理を組み合わせることで実現しています。

- **`app.py`**: アプリケーションのエントリーポイント。チャット機能、履歴閲覧、サンプルデータ管理のUIを提供します。
- **`ui.py`**: チャットページや履歴閲覧ページなど、アプリケーションのUIロジックを管理します。
- **`llm.py`**: LLMモデルのロードとテキスト生成を行うモジュール。
- **`database.py`**: SQLiteデータベースを使用してチャット履歴やフィードバックを保存・管理します。
- **`metrics.py`**: BLEUスコアやコサイン類似度など、回答の評価指標を計算するモジュール。
- **`data.py`**: サンプルデータの作成やデータベースの初期化を行うモジュール。
- **`config.py`**: アプリケーションの設定（モデル名やデータベースファイル名）を管理します。
- **`requirements.txt`**: このアプリケーションを実行するために必要なPythonパッケージ。

後片付けとして、使う必要のないngrokのトンネルを削除します。

In [ ]:
from pyngrok import ngrok
ngrok.kill()

In [ ]:
!ls

app_phi.py  chat_feedback.db  database.py  llm.py      __pycache__	 ui.py
app.py	    config.py	      data.py	   metrics.py  requirements.txt


In [ ]:
import sqlite3
conn = sqlite3.connect("chat_feedback.db")
c = conn.cursor()

c.execute("SELECT * FROM chat_history")
rows = c.fetchall()
for row in rows:
    print(row)

conn.close()

(1, '2025-04-19 04:25:20', 'Pythonのリスト内包表記とは何ですか？', 'リスト内包表記は、既存のリストから新しいリストを作成するためのPythonの構文です。通常のfor文よりも簡潔に記述でき、パフォーマンスも向上する場合があります。', '部分的に正確: 基本的な説明は正しいですが、具体的な構文例が示されていません', 'Pythonのリスト内包表記は、リストを簡潔に作成するための構文で、`[expression for item in iterable if condition]`の形式で書きます。通常のforループよりも短く書けて、実行速度も速い場合があります。', 0.5, 1.2, 0.0, 0.0, 41, 0.0)
(2, '2025-04-19 04:25:20', '機械学習における過学習とは？', '過学習（オーバーフィッティング）とは、機械学習モデルが訓練データに対して過度に適合し、新しいデータに対する汎化性能が低下する現象です。', '正確: 過学習の本質をよく捉えています', '過学習（オーバーフィッティング）は、モデルがトレーニングデータに過度に適合し、未知のデータに対する予測性能が低下する現象です。モデルが訓練データのノイズまで学習してしまうことが原因です。', 1.0, 1.5, 0.0, 0.25233420143369617, 32, 0.3333333333333333)
(3, '2025-04-19 04:25:21', '量子コンピュータの基本原理は？', '量子コンピュータは量子力学の原理に基づいて動作します。従来のビットの代わりに量子ビット（キュービット）を使用し、重ね合わせと量子もつれの特性により並列計算を実現します。', '部分的に正確: 基本概念は正しいですが、詳細な説明が不足しています', '量子コンピュータは量子力学的現象を利用した計算機で、従来のビットではなく量子ビット（キュービット）を使用します。キュービットは重ね合わせ状態をとることができ、複数の状態を同時に表現できます。また量子もつれにより、従来のコンピュータでは困難な特定の問題を効率的に解くことができます。', 0.5, 2.1, 0.0, 0.08704446792504217, 44, 0.125)
(4, '2

# 03_FastAPI

ディレクトリ「03_FastAPI」に移動します。

In [5]:
%cd /content/lecture-ai-engineering/day1/03_FastAPI

/content/lecture-ai-engineering/day1/03_FastAPI


必要なライブラリをインストールします。

In [6]:
%%capture
!pip install -r requirements.txt

ngrokとhuggigfaceのトークンを使用して、認証を行います。

In [7]:
!ngrok authtoken $$NGROK_TOKEN
!huggingface-cli login --token $$HUGGINGFACE_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `agent` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `agent`


アプリを起動します。

「02_streamlit_app」から続けて「03_FastAPI」を実行している場合は、モデルのダウンロードが済んでいるため、すぐにサービスが立ち上がります。

「03_FastAPI」のみを実行している場合は、初回の起動時にモデルのダウンロードが始まるので、モデルのダウンロードが終わるまで数分間待ちましょう。

### Gemma 2

In [ ]:
!python app.py

### Phi 4

promptに以下を加えた。
- 答えを教えるのではなく、一緒に考えるように促すようにしてください。
- 会話は質問で終わるようにしてください。

Rather than just providing an answer, try to guide the user to think with you. Always finish your response with a question.

In [ ]:
!python app_phi.py

2025-04-20 14:40:49.990016: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-20 14:40:50.007871: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745160050.029863    2444 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745160050.036437    2444 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-20 14:40:50.058720: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

FastAPIが起動すると、APIとクライアントが通信するためのURL（エンドポイント）が作られます。

URLが作られるのと合わせて、Swagger UIというWebインターフェースが作られます。

Swagger UIにアクセスすることで、APIの仕様を確認できたり、APIをテストすることができます。

Swagger UIを利用することで、APIを通してLLMを動かしてみましょう。

後片付けとして、使う必要のないngrokのトンネルを削除します。

In [11]:
from pyngrok import ngrok
ngrok.kill()